In [8]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'sd-4sql\\packages'))
from query_parser import *
from moz_sql_parser import parse

In [ ]:
query1 = 'SELECT ik FROM VT_Article ORDER BY datMod DESC'
print(query1)
parsing(query1)

### Query with Join clause and functions

In [27]:
query = 'select count( distinct a.ik) from fr.infologic.ventes.commandesfactures.modele.CdeLig as a where a.cde.typCde = :p1 and a.typLigGenere != :p2 and a.typLigGenere != :p3 and a.art.refDefaut = :p4 and (a.cde.cliLiv = :p5 or 9596436491 in elements (a.cde.noCliContrat)) and ( a.cde.etatContrat in ( :collection0_ ) ) and ( a.etatContrat NOT in ( :collection1_ ) ) and UPPER ( a.libStd ) like UPPER( :p6 ) and a.dossierInfo.dosRes = :p7'
parsing(query)

here


{'tables_from': ['fr.infologic.ventes.commandesfactures.modele.cdelig'],
 'tables_join': [],
 'projections': ['fr.infologic.ventes.commandesfactures.modele.cdelig.ik'],
 'atts_where': ['fr.infologic.ventes.commandesfactures.modele.cdelig.cde.typcde',
  'fr.infologic.ventes.commandesfactures.modele.cdelig.typliggenere',
  'fr.infologic.ventes.commandesfactures.modele.cdelig.typliggenere',
  'fr.infologic.ventes.commandesfactures.modele.cdelig.art.refdefaut',
  'fr.infologic.ventes.commandesfactures.modele.cdelig.cde.cliliv',
  'fr.infologic.ventes.commandesfactures.modele.cdelig.cde.noclicontrat',
  'fr.infologic.ventes.commandesfactures.modele.cdelig.cde.etatcontrat',
  'fr.infologic.ventes.commandesfactures.modele.cdelig.etatcontrat',
  'fr.infologic.ventes.commandesfactures.modele.cdelig.libstd',
  'fr.infologic.ventes.commandesfactures.modele.cdelig.dossierinfo.dosres'],
 'atts_groupby': [],
 'atts_orderby': [],
 'atts_having': [],
 'functions': ['count']}

In [30]:
query = 'select distinct a.dossierInfo.dosRes, a.rgt.elements.ancienModRgt.dossierInfo.dosRes from fr.infologic.ventes.commandesfactures.modele.HistFacture a'
parsing(query)

{'tables_from': ['fr.infologic.ventes.commandesfactures.modele.histfacture'],
 'tables_join': [],
 'projections': ['fr.infologic.ventes.commandesfactures.modele.histfacture.dossierinfo.dosres',
  'fr.infologic.ventes.commandesfactures.modele.histfacture.rgt.elements.ancienmodrgt.dossierinfo.dosres'],
 'atts_where': [],
 'atts_groupby': [],
 'atts_orderby': [],
 'atts_having': [],
 'functions': []}

In [26]:
def parsing(query) :
    
    query = query.lower()
    
    if ('xmlelement' in query) or ('b.prod=' in query) or ('as float' in query) or len(query) > 120000 :    
        print('NOT SUPPORTED')
        return 0
    else : 
        try : 
            query = preprocessing(query)
            query_parsed = {}

            query_reg = parse(query) 
            alias = get_tables(query_reg,{})[1]
            query_parsed['tables_from']  =  get_tables_fj(query_reg, {})[0]
            query_parsed['tables_join']  =  get_tables_fj(query_reg, {})[1]
            query_parsed['projections']  =  get_projections(query_reg, alias)
            query_parsed['atts_where']   =  get_atts_where(query_reg, alias)
            query_parsed['atts_groupby'] =  get_atts_group_by(query_reg, alias)
            query_parsed['atts_orderby'] =  get_atts_order_by(query_reg, alias)
            query_parsed['atts_having']  =  get_atts_having(query_reg, alias)
            query_parsed['functions']    =  get_functions(query_reg)

        except :
            print(query)
        return query_parsed

In [25]:
def preprocessing(query) :
         
    query = query.replace(':', '')
    query = query.replace('?','0')
    query = query.replace(' new','')
    query = query.replace('join fetch','join')
    query = query.replace('where exists (','where exists ( select * ')
    query = query.replace('[ 0 ','')
    query = query.replace(']','')
    query = query.replace('mon,','mon join ')
    query = query.replace('(from','(select * from')
    query = query.replace('= :p', '= p')
    query = query.replace(':collection', 'collection')
    query = query.replace('elements ', 'elements')
    
    if ('order by' in query) and ('group by' in query) and query.find('order by') < query.find('group by') :
        query = swap_orderby_groupby(query)
    
    if (', fr.infologic' in query) and ('join' in query) and query.find(', fr.infologic') > query.find('join') :
        query = query.replace(', fr.infologic',' join fr.infologic')
    
    if 'elements (' in query or 'elements(' in query:
        print('here')
        query = transform_elements(query)
    
    return query

In [4]:
query2 = 'select count(*) from fr.infologic.ventes.commandesfactures.modele.commande as cde inner join cde.cdeedi as cdeedi inner join cdeedi.messagerecu as m left join cdeedi.tiersediintegration as partedi left join cdeedi.cliliv as liva left join cdeedi.clilivintegration as partediinteg left join cdeedi.pdaintegration as pdainteg left join cdeedi.clicde as cdepar left join cde.repres1 as repres1 left join cdeedi.regleintegrationcde as regleinteg left join cde.degroup as degroup left join cde.transp as cdetransp left join cdeedi.cdeediglobal as cdeediglobal left join liva.operateur as opcli left join opcli.util as utopcli left join cde.siteexped.site as site where cdeedi.dossierinfo.dosres = dossieredi and cde.dossierinfo.dosres = dossiercde and cdeedi.typcdeedi in (types) and ((cde.datdepart >= datdepartmini or cde.datdepart is null) or (cde.typcde = 7 and cde.blocageedianomalie = 1 and cdeedi.datdepartmaxi >= datdepartmini)) and ((cde.datdepart <= datdepartmaxi or cde.datdepart is null) or (cde.typcde = 7 and cde.blocageedianomalie = 1 and cdeedi.datdepartmini <= datdepartmaxi)) and cde.typcde in (typescde) and cdeedi.etatcde in (etatcde)'
print(query2)
parsing(query2)

select count(*) from fr.infologic.ventes.commandesfactures.modele.commande as cde inner join cde.cdeedi as cdeedi inner join cdeedi.messagerecu as m left join cdeedi.tiersediintegration as partedi left join cdeedi.cliliv as liva left join cdeedi.clilivintegration as partediinteg left join cdeedi.pdaintegration as pdainteg left join cdeedi.clicde as cdepar left join cde.repres1 as repres1 left join cdeedi.regleintegrationcde as regleinteg left join cde.degroup as degroup left join cde.transp as cdetransp left join cdeedi.cdeediglobal as cdeediglobal left join liva.operateur as opcli left join opcli.util as utopcli left join cde.siteexped.site as site where cdeedi.dossierinfo.dosres = dossieredi and cde.dossierinfo.dosres = dossiercde and cdeedi.typcdeedi in (types) and ((cde.datdepart >= datdepartmini or cde.datdepart is null) or (cde.typcde = 7 and cde.blocageedianomalie = 1 and cdeedi.datdepartmaxi >= datdepartmini)) and ((cde.datdepart <= datdepartmaxi or cde.datdepart is null) or (c

{'tables_from': ['fr.infologic.ventes.commandesfactures.modele.commande'],
 'tables_join': ['fr.infologic.ventes.commandesfactures.modele.commande.cdeedi',
  'fr.infologic.ventes.commandesfactures.modele.commande.cdeedi.messagerecu',
  'fr.infologic.ventes.commandesfactures.modele.commande.cdeedi.tiersediintegration',
  'fr.infologic.ventes.commandesfactures.modele.commande.cdeedi.cliliv',
  'fr.infologic.ventes.commandesfactures.modele.commande.cdeedi.clilivintegration',
  'fr.infologic.ventes.commandesfactures.modele.commande.cdeedi.pdaintegration',
  'fr.infologic.ventes.commandesfactures.modele.commande.cdeedi.clicde',
  'fr.infologic.ventes.commandesfactures.modele.commande.repres1',
  'fr.infologic.ventes.commandesfactures.modele.commande.cdeedi.regleintegrationcde',
  'fr.infologic.ventes.commandesfactures.modele.commande.degroup',
  'fr.infologic.ventes.commandesfactures.modele.commande.transp',
  'fr.infologic.ventes.commandesfactures.modele.commande.cdeedi.cdeediglobal',
  'f

### Nested Query

In [ ]:
query3 = 'select bp from fr.infologic.ventes.commandesfactures.modele.histbonlivposition as bp join bp.bonliv as bonliv where bp.position.bordereau.dossierinfo.dosres =0 and bonliv.datdepart <=0 and bonliv.datdepart >=0 and bp.typbordereau >0 and bp.typbordereau in ( select max(b.typbordereau) from fr.infologic.ventes.commandesfactures.modele.histbonlivposition as b join b.bonliv as a where a.ik = bonliv.ik ) and bonliv.transp in(transportlist) order by bonliv.transp.ik, bonliv.degroup.ik, bonliv.degroup2.ik, bonliv.degroup3.ik'
print(query3)
parsing(query3)

In [ ]:
'------------------------------------------------------------------------------------------------------------------------'

In [ ]:
saved_path = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))),'Data\\saved-data\\')
queries = pd.read_csv(saved_path + 'requetes_hib.csv')

In [ ]:
queries[queries['requete'].str.contains('elements')]['requete'].values[0]

In [ ]:
query = 'select count( distinct a.ik) from fr.infologic.ventes.commandesfactures.modele.CdeLig as a where a.cde.typCde = :p1 and a.typLigGenere != :p2 and a.typLigGenere != :p3 and a.art.refDefaut = :p4 and (a.cde.cliLiv = :p5 or 9596436491 in elements (a.cde.noCliContrat)) and ( a.cde.etatContrat in ( :collection0_ ) ) and ( a.etatContrat NOT in ( :collection1_ ) ) and UPPER ( a.libStd ) like UPPER( :p6 ) and a.dossierInfo.dosRes = :p7'
print(query)
parsing(query)